In [2]:
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import plotly.express as px
import re
import streamlit as st

In [3]:
from matplotlib.image import thumbnail


ambcass = pd.read_csv('data/Aggregated_Metrics_By_Country_And_Subscriber_Status.csv')
ambv = pd.read_csv('data/Aggregated_Metrics_By_Video.csv').iloc[1:]
acv = pd.read_csv('data/All_Comments_Final.csv')
vpot = pd.read_csv('data/Video_Performance_Over_Time.csv')

vpot['Date'] = pd.to_datetime(vpot['Date'])
df = pd.merge(vpot, vpot.groupby('Video Title')['Date'].agg('min'), left_on='Video Title', right_index=True, how='left')
df = df.rename(columns={'Date_x': 'Date', 'Date_y':'Start Date'})
df['Days from start'] = df['Date'] - df['Start Date']
df['Days from start'] = df['Days from start'].dt.days

ambv.columns = list(map(lambda x:re.sub(r'[^a-zA-Z0-9\(\)\%\- ]', '', x), ambv.columns))
ambv = ambv.astype({'Video publish time':'datetime64[ns]',
                   'Average view duration':'datetime64[ns]'})
ambv['Average view duration'] = ambv['Average view duration'].dt.minute * 60 + ambv['Average view duration'].dt.second
thumbnail_links = ambcass[['External Video ID', 'Thumbnail link']].drop_duplicates().reset_index(drop=True)
ambv = pd.merge(ambv, thumbnail_links, left_on='Video', right_on='External Video ID', how='left').drop('External Video ID', axis=1)


In [4]:
titles = ambcass['Video Title'].unique()[::40]

In [5]:
titles

array(['🌶 Hot Topics in Tech: Data Science Explained #SHORTS',
       "The Secret Data Scientists Don't Want You to Know",
       'Reviewing Your Data Science Projects - Episode 19 (One Big Improvement)',
       'I Eat a Papaya Live on Stream (Plus Q&A for 150K Subs!)',
       'Fast Cars to Faster Data (Alex Castrounis) - KNN EP. 12',
       'Beginner Kaggle Data Science Project Walk-Through (Titanic)'],
      dtype=object)

In [10]:
kek = ambcass[ambcass['Video Title'].isin(titles)].groupby('Country Code')['Views'].mean()
kek.sort_values(ascending=False)

Country Code
US    3352.416667
IN    3345.166667
GB     767.000000
DE     610.583333
CA     588.250000
         ...     
TL       1.000000
ST       1.000000
BI       0.750000
MF       0.000000
GL       0.000000
Name: Views, Length: 204, dtype: float64